# 预测导向的算法

在本章的学习中，我们将介绍使用交互协变量的监督学习，这些数据驱动的协变量能够更灵活地捕捉数据中的特质。但是，由于其过于复杂，我们很难对这些方法给出详细的理论解释。因此，理论学者把它们这种难以解释的计算过程称为“黑箱”。在实际应用中，如果调试得当，那么它们的预测效果会十分显著。
{cite}`gu2018empirical` 展示了用交互项辅助金融市场预测的一系列方法。但与此同时，由于它实际上并不符合常规的科学范式，业界人士也在当心这些方法是否过于“玄学”。在真正将这些方法应用于计量分析前，我们必须先谨慎地对它们进行验证。

## 回归树和装袋法

监督学习实际上就是用x来预测y，非参数估计(例如核回归)就可以进行计算。此外还有一种方法是 **回归树** [{cite}`breiman1984classification`] 。 回归树本质上是递归地拆分统计量的空间。背后的算法其实不难理解：每一次计算都将一个协变量拆分为两个虚拟变量。而依据的标准则是残差平方和(SSR)是否大幅降低。每一次拆分的拟合值都用 $y_i$ 残差平方和的均值来计算。

回归树的调谐参数是它的深度，也就是拆分的次数。给定一个数据集 $d$ 和回归树的深度，就能用数据拟合的回归树 $\hat{r}(d)$ 。


回归树的问题在于它的不稳定性。对于同一个DGP，选择不同的协变量和分裂点可能会极大地影响拆分的路径。

**自举平均法** ，也叫 **装袋法** 可以用于减少回归树的方差[{cite}`breiman1996bagging`]。 装袋法在每一个自举样本中都建立一个回归树，然后进行简单的算数平均。如果 $d^{\star b}$ 是原数据 $d$ 的 $b$ 阶自举样本，我们可以将装袋法估计量定义为

$$
\hat{r}_{\mathrm{bagging}} = B^{-1} \sum_{b=1}^B \hat{r}( d^{\star b} ).
$$

(装袋法是一种 **集成学习**)

{cite}`inoue2008useful` 演示了装袋法在时间序列预测中的应用。
{cite}`hirano2017forecasting` 提供了一个减少袋装法风险的新理论视角。


## 随机森林

**随机森林** [{cite}`breiman2001random`] 在总体 $p$ 中随机抽取样本 $m$ 来构造 **子树** 进行分类。
随机森林中的调谐参数是树的深度和 $m$ 。 由于抽样回归的“去相关性”，一般情况下它的预测效果比装袋法更好。

下方是使用波士顿住房数据的随机森林的简单例子：

In [ ]:
require(randomForest)
require(MASS) # Package which contains the Boston housing dataset
attach(Boston)
set.seed(101)

# training Sample with 300 observations
train <- sample(1:nrow(Boston), 300)

Boston.rf <- randomForest(medv ~ ., data = Boston, subset = train)
plot(Boston.rf)

importance(Boston.rf)


尽管算法十分简单，随机森林的相合性直到2015年才被{cite}`scornet2015consistency`证明；而推断理论则是由{cite}`wager2018estimation`首先提出；{cite}`athey2019generalized` 将 CART 推广到局部极大似然。

**例子**：{cite}`cheng2020survey`的 `data_example/SPF_RF.R` 能够进行专业的随机森林预测(该脚本使用的框架是`caret`)。 

## 梯度提升

装袋法和随机森林在生成树时总是使用相同的权重。然而，**提升树** 法用一种特别的方式来决定权重。它是一种确定性方法，不对原数据进行重抽样。

1. 用原数据 $d^0=(x_i,y_i)$ 生成一个影树 $\hat{r}^{0}(d^0)$ 。保留预测结果 $f^0_i = \alpha \cdot \hat{r}^0 (d^0, x_i)$ (其中
$\alpha\in [0,1]$ 是一个压缩调谐参数)和残差 $e_i^{0} = y_i - f^0_i$ ，令 $m=1$ 。
2. 在第 $m$ 次迭代中, 使用数据 $d^m = (x_i,e_i^{m-1})$ 生成影树 $\hat{r}^{m}(d^m)$ 。保留预测结果 $f^m_i =  f^{m-1}_i +  \alpha \cdot \hat{r}^m (d, x_i)$ 和残差 $e_i^{m} = y_i - f^m_i$ 。使 $m = m+1$ 。
3.重复步骤2直到 $m > M$.

在这个提升算法中有三个调谐参数：树的深度、压缩水平 $\alpha$ 和迭代次数 $M$ 。
算法对于三个参数都很敏感，当参数设定合理时，最终效果会非常显著。例如{cite}`lin2020`指出，脚本 `Beijing_housing_gbm.R` 得到的样本外 $R^2$ 比OLS显著高出不少。该脚本使用 `gbm` (Gradient Boosting Machine) 包进行梯度提升运算。

Boosting算法不只这一种。例如 $L_2$ 提升、成分提升和适应性提升等等。统计学者们将Boosting视为梯度下降算法来降低风险。每次迭代中拟合的树的下降速度都是最快的，而压缩会放缓拟合的速度来避免降低幅度过快。

* {cite}`shi2016econometric` 提出了一种与Boosting思路类似的greedy算法，并用来挑选GMM中的矩条件。
* {cite}`phillips2019boosting` 用 $L_2$ 提升作为Hodrick-Prescott滤波的进阶版。
* {cite}`shi2019forward`

## 神经网络

神经网络是Alpha-Go和自动驾驶背后的支撑技术。但在统计学者看来，它只不过是一种特殊的非线性模型。
图1展示了一个单层的神经网络。不过，大多数情况下神经网络都不只一层，从 $k-1$ 层到 $k$ 层的过渡可以写成

$$
\begin{align*}
z_l^{(k)} & =  w_{l0}^{(k-1)} + \sum_{j=1}^{p_{k-1} } w_{lj}^{(k-1)} a_j^{(k-1)} \\ 
a_l^{(k)} & =  g^{(k)} ( z_l^{(k)}).
\end{align*}
$$

其中 $a_j^{(0)}  = x_j$  是输入值,  $z_l^{(k)}$ 是第 $k$ 个隐藏层，当中所有的 $w$ 都是要估计的系数。
上述公式表明 $z_l^{(k)}$ 通常是线性形式，**激励函数** $g(\cdot)$ 是恒等函数或者简单的非线性函数。
激励函数通常选择 sigmoid 函数 ($1/(1+\exp(-x))$) 或者线性整流函数 (ReLu, $z\cdot 1\{x\geq 0\}$)。


![A Single Layer Feedforward Neural Network (from Wiki)](graph/Colored_neural_network.png)

拟合神经网络通常要考虑几个因素：除激励函数外，隐藏层的数量和每一层的节点也是重要的调谐参数。每一层和每一个节点都会生成许多的自由参数，然后可以用正则化方法对于 $l_1$ 与 $l_2$ 规范对其进行惩罚。
`data_example/Keras_ANN.R` 给出一个两层隐藏层的神经网络的例子(每层有64个节点)，并使用了线性整流激励函数。

由于神经网络本身是非线性，目前对其尚未有明确的理论解释。早期的一个理论贡献来自于{cite}`hornik1989multilayer`的定理2.2：给定足够多的节点，一个单层神经网络是任何可测函数的**万能近似器**。

在建立一个神经网络后，自由参数必须要用数值最优化决定。非线性的复杂结构使得最优化极具挑战性，我们很难能够得到全局的最优值。尤其当样本数量较大时，最优化算法是随机梯度下降的。 

多亏了计算科学家， 谷歌的 `tensorflow` 是一个深受欢迎的神经网络后端， `keras` 是深度学习建模语言。
它们之间的关系类似于 `Mosek` 和 `CVXR` 。

## 随机梯度下降

在最优化过程中， 我们迭代 $D$ 维参数

$$
\beta_{k+1} = \beta_{k} + a_k p_k,
$$

其中 $a_k \in \mathbb{R}$ 是步长， $p_k\in \mathbb{R}^D$ 是方向向量。使用泰勒级数展开可以得到：

$$
f(\beta_{k+1}) = f(\beta_k + a_k p_k ) \approx f(\beta_k) + a_k \nabla f(\beta_k) p_k,
$$

如果我们希望目标函数 $f(x)$ 在每一步中递减，那么需要 $\nabla f(\beta_k) p_k \leq 0$ 。
我们可以令 $p_k =-\nabla f(\beta_k)$ ，这种方式通常称为**最深下降**(the deepest descent)。
牛顿法则是令 $p_k =- (\nabla^2 f(\beta_k))^{-1}  \nabla f(\beta_k)$ ，
然后 BFGS 用一个低阶矩阵来估计 $\nabla^2 f(\beta_k)$ 。 线性搜索是一维问题，可以通过精确最小化或回溯来解决。下降法的具体细节可以参考  {cite}`boyd2004convex` 的 9.2--9.5 章节。

当样本规模大且参数数量过多时，梯度的求解将会非常的复杂。随机梯度下降 (SGD) 在每次迭代中都只用一小部分的样本估计梯度，这样可以大大减少计算的时间。 它是神经网络训练这种复杂的最优化问题中的 **事实** 最优化过程。

不过， SGD 涉及到了调谐参数 (比如batch的大小和学习速率，学习速率代替了步长 $a_k$ 并变成了正则参数)。这些参数将会极大地影响最终结果，这种情况在非线性问题中更加显著。因此，在真正实操之前，算法必须经过仔细的测试。


下面是一个PPMLE中SGD的例子，在最优化那节课中也提到过这个例子。在这里它的样本规模是100，000且参数数量为100。通常情况下， SGD 比 `nlopt`要快很多。

由于在SGD中每次的对数似然方程和梯度都是在一个不同的子样本中计算的，所以在此我们将带有数据的新函数定义为arguments。


In [ ]:
poisson.loglik <- function(b, y, X) {
  b <- as.matrix(b)
  lambda <- exp(X %*% b)
  ell <- -mean(-lambda + y * log(lambda))
  return(ell)
}


poisson.loglik.grad <- function(b, y, X) {
  b <- as.matrix(b)
  lambda <- as.vector(exp(X %*% b))
  ell <- -colMeans(-lambda * X + y * X)
  ell_eta <- ell
  return(ell_eta)
}


In [ ]:
##### generate the artificial data
set.seed(898)
nn <- 1e5
K <- 100
X <- cbind(1, matrix(runif(nn * (K - 1)), ncol = K - 1))
b0 <- rep(1, K) / K
y <- rpois(nn, exp(X %*% b0))

b.init <- runif(K)
b.init <- 2 * b.init / sum(b.init)
# and these tuning parameters are related to N and K

n <- length(y)
test_ind <- sample(1:n, round(0.2 * n))

y_test <- y[test_ind]
X_test <- X[test_ind, ]

y_train <- y[-test_ind ]
X_train <- X[-test_ind, ]

In [ ]:
# optimization parameters
# sgd depends on

# * eta: the learning rate
# * epoch: the averaging small batch
# * the initial value

max_iter <- 5000
min_iter <- 20
eta <- 0.01
epoch <- round(100 * sqrt(K))

b_old <- b.init

pts0 <- Sys.time()
# the iteration of gradient
for (i in 1:max_iter) {
  loglik_old <- poisson.loglik(b_old, y_train, X_train)
  i_sample <- sample(1:length(y_train), epoch, replace = TRUE)
  b_new <- b_old - eta * poisson.loglik.grad(b_old, y_train[i_sample], X_train[i_sample, ])
  loglik_new <- poisson.loglik(b_new, y_test, X_test)
  b_old <- b_new # update

  criterion <- loglik_old - loglik_new

  if (criterion < 0.0001 & i >= min_iter) break
}
cat("point estimate =", b_new, ", log_lik = ", loglik_new, "\n")
pts1 <- Sys.time() - pts0
print(pts1)


# optimx is too slow for this dataset.
# Nelder-Mead method is too slow for this dataset

# thus we only sgd with NLoptr

opts <- list("algorithm" = "NLOPT_LD_SLSQP", "xtol_rel" = 1.0e-7, maxeval = 5000)


pts0 <- Sys.time()
res_BFGS <- nloptr::nloptr(
  x0 = b.init,
  eval_f = poisson.loglik,
  eval_grad_f = poisson.loglik.grad,
  opts = opts,
  y = y_train, X = X_train
)
pts1 <- Sys.time() - pts0
print(pts1)

b_hat_nlopt <- res_BFGS$solution

#### evaluation in the test sample
cat("log lik in test data by sgd = ", poisson.loglik(b_new, y = y_test, X_test), "\n")
cat("log lik in test data by nlopt = ", poisson.loglik(b_hat_nlopt, y = y_test, X_test), "\n")
cat("log lik in test data by true para. = ", poisson.loglik(b0, y = y_test, X_test), "\n")


## 拓展阅读

* Efron and Hastie: 第 8、 17 和 18章节

## 引言

<!-- "The world is yours, as well as ours, but in the last analysis, it is yours. You young people, full of vigor and vitality, are in the bloom of life, like the sun at eight or nine in the morning. Our hope is placed on you." -->

<!-- ---Mao Zedong, Talk at a meeting with Chinese students and trainees in Moscow (November 17, 1957). -->

## 参考文献